In [14]:
! pip show torch
! pip install torchsummary

Name: torch
Version: 1.4.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/ali/.virtualenvs/py37/lib/python3.7/site-packages
Requires: 
Required-by: torchvision, timm, pretrainedmodels, efficientnet-pytorch, catalyst
  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


In [3]:
from  torchvision import models
from torchsummary import summary
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

resnet50=models.resnet50(pretrained=True).to(device)
vgg = models.vgg16().to(device)
# summary(vgg, (3, 224, 224))
summary(resnet50, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class My_model(nn.Module):
    def __init__(self):
        super(My_model, self).__init__()
        resnet50 = models.resnet50(pretrained=True)
        self.resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1]))
        
        for param in self.resnet50.parameters():
            param.requires_grad = False

        self.encode_front_bbox =  nn.Sequential(
            torch.nn.Linear(4, 256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=.25),
            torch.nn.Linear(256, 256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=.25),            
            torch.nn.Linear(256, 256),
            torch.nn.ReLU(inplace=True)           
         )
        
        self.decode_to_birdeye_bbox =  nn.Sequential(
            torch.nn.Linear(256+2048, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=.25),
            torch.nn.Linear(1024, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=.25), 
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=.25), 
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=.25), 
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=.25),
            
            torch.nn.Linear(128, 4),
            torch.nn.Tanh(),
           
         )
#         self.classifier = nn.Linear(4, 2)
#         self.la
        
    def forward(self, croped_f_image , f_bbox):
        batch = croped_f_image.shape[0]
        x1 = self.resnet50(croped_f_image)
        x1 = x1.view(batch, 2048)
        
        encoded_f_bbox = self.encode_front_bbox(f_bbox)
        x1 = torch.cat((x1, encoded_f_bbox), dim=1)
        out = self.decode_to_birdeye_bbox (x1)
        
        
        
#         x1 = self.norm_layer(x1)
#         x = torch.cat((x1, x2), dim=1)
#         x = self.classifier(F.relu(x))
        return out

In [5]:
# print(My_model())
# summary(My_model().to(device), (3, 224, 224) )

model = My_model()
t_in = torch.randn(1,3,224,224 )
t_in2 = torch.randn(1,4)
model(t_in,t_in2).shape

# print(model)

torch.Size([1, 4])

In [24]:
# for name, child in model.named_children():
#     for name2, params in child.named_parameters():
#         print(name, name2)

In [60]:
class MyEnsemble(nn.Module):
    def __init__(self, modelA):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.norm_layer = nn.AdaptiveAvgPool2d(1)
#         self.classifier = nn.Linear(4, 2)
#         self.la
        
    def forward(self, x1):
        x1 = self.modelA(x1)
        x1 = self.norm_layer(x1)
#         x = torch.cat((x1, x2), dim=1)
#         x = self.classifier(F.relu(x))
        return x1
  
    
# model = models.resnet152(pretrained=True)
resnet50 = models.resnet50(pretrained=True)
resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1])).to(device)
# print(newmodel)
# summary(resnet50, (3, 224, 224))    
    
    
    
resnet50 = MyEnsemble(resnet50)
summary(resnet50, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [76]:
resnet50 = models.resnet50(pretrained=True)

print(torch.nn.Sequential(*(list(resnet50.children())[-9])))

TypeError: type object argument after * must be an iterable, not BatchNorm2d

In [166]:
! pip install graphviz
! pip install  torchviz

  Using cached torchviz-0.0.1.tar.gz (41 kB)
  Created wheel for torchviz: filename=torchviz-0.0.1-py3-none-any.whl size=3522 sha256=b203e37b3e9a7d035344d62b2d556e31b1ca0ba414a937fa61a143742c7d5959
  Stored in directory: /home/ali/.cache/pip/wheels/10/7b/c8/3af79ec02e294a832c01037bcb38302bbcee0bb020dcbbbd3e
Successfully built torchviz


In [171]:
from graphviz import Digraph
import torch
from torch.autograd import Variable


def make_dot(var, params):
    """ Produces Graphviz representation of PyTorch autograd graph
    
    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function
    
    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    param_map = {id(v): k for k, v in params.items()}
    print(param_map)
    
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
    
    def size_to_str(size):
        return '('+(', ').join(['%d'% v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                node_name = '%s\n %s' % (param_map.get(id(u)), size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot

from torchvision import models
inputs = torch.randn(1,3,224,224)
resnet18 = models.resnet18()
y = resnet18(Variable(inputs))
# print(y)

g = make_dot(y, resnet18.state_dict())
g.view()

{139651348686960: 'conv1.weight', 139649785527184: 'bn1.weight', 139649785527984: 'bn1.bias', 139649785527904: 'bn1.running_mean', 139649783774256: 'bn1.running_var', 139649752169360: 'bn1.num_batches_tracked', 139649858835152: 'layer1.0.conv1.weight', 139649858834672: 'layer1.0.bn1.weight', 139651349476848: 'layer1.0.bn1.bias', 139651349476928: 'layer1.0.bn1.running_mean', 139651349476448: 'layer1.0.bn1.running_var', 139649783383216: 'layer1.0.bn1.num_batches_tracked', 139649785145456: 'layer1.0.conv2.weight', 139651350426880: 'layer1.0.bn2.weight', 139651306555984: 'layer1.0.bn2.bias', 139651435579584: 'layer1.0.bn2.running_mean', 139651306844240: 'layer1.0.bn2.running_var', 139649757753824: 'layer1.0.bn2.num_batches_tracked', 139651306768096: 'layer1.1.conv1.weight', 139649783353584: 'layer1.1.bn1.weight', 139649784811344: 'layer1.1.bn1.bias', 139651306486192: 'layer1.1.bn1.running_mean', 139651309028048: 'layer1.1.bn1.running_var', 139649752180848: 'layer1.1.bn1.num_batches_tracked

'Digraph.gv.pdf'

In [173]:
# inputs = torch.randn(1,3,224,224)
# resnet18 = models.resnet18()
# y = resnet18(Variable(inputs))
# # print(y)

# g = make_dot(y, resnet18.state_dict())
# g.view()

t_in = torch.randn(1,3,224,224 )
t_in2 = torch.randn(1,4)
y = model(Variable(t_in),Variable(t_in2))

g = make_dot(y, model.state_dict())
g.view()

{139649783651696: 'resnet50.0.weight', 139649764723760: 'resnet50.1.weight', 139649764722880: 'resnet50.1.bias', 139649764724160: 'resnet50.1.running_mean', 139649764721440: 'resnet50.1.running_var', 139649764722080: 'resnet50.1.num_batches_tracked', 139649764724640: 'resnet50.4.0.conv1.weight', 139649764722720: 'resnet50.4.0.bn1.weight', 139649764722400: 'resnet50.4.0.bn1.bias', 139649764723200: 'resnet50.4.0.bn1.running_mean', 139649764720720: 'resnet50.4.0.bn1.running_var', 139649764721200: 'resnet50.4.0.bn1.num_batches_tracked', 139649764669712: 'resnet50.4.0.conv2.weight', 139649764669952: 'resnet50.4.0.bn2.weight', 139649764669872: 'resnet50.4.0.bn2.bias', 139649764669392: 'resnet50.4.0.bn2.running_mean', 139649764668912: 'resnet50.4.0.bn2.running_var', 139649764669072: 'resnet50.4.0.bn2.num_batches_tracked', 139649764670112: 'resnet50.4.0.conv3.weight', 139649764671232: 'resnet50.4.0.bn3.weight', 139649764668592: 'resnet50.4.0.bn3.bias', 139649764670272: 'resnet50.4.0.bn3.runnin

'Digraph.gv.pdf'

In [177]:
!pip install ipywidgets

  Using cached ipywidgets-7.5.1-py2.py3-none-any.whl (121 kB)
  Using cached nbformat-5.0.4-py3-none-any.whl (169 kB)
  Using cached widgetsnbextension-3.5.1-py2.py3-none-any.whl (2.2 MB)
  Using cached jsonschema-3.2.0-py2.py3-none-any.whl (56 kB)
  Using cached notebook-6.0.3-py3-none-any.whl (9.7 MB)
  Using cached attrs-19.3.0-py2.py3-none-any.whl (39 kB)
  Using cached pyrsistent-0.15.7.tar.gz (107 kB)
     |████████████████████████████████| 126 kB 10.1 MB/s eta 0:00:01
  Using cached nbconvert-5.6.1-py2.py3-none-any.whl (455 kB)
  Using cached Send2Trash-1.5.0-py3-none-any.whl (12 kB)
  Using cached prometheus_client-0.7.1.tar.gz (38 kB)
  Using cached terminado-0.8.3-py2.py3-none-any.whl (33 kB)
  Using cached testpath-0.4.4-py2.py3-none-any.whl (163 kB)
  Using cached entrypoints-0.3-py2.py3-none-any.whl (11 kB)
  Using cached mistune-0.8.4-py2.py3-none-any.whl (16 kB)
  Using cached defusedxml-0.6.0-py2.py3-none-any.whl (23 kB)
     |████████████████████████████████| 150 kB 28

In [1]:
from ipywidgets import IntProgress
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter


resnet18 = models.resnet18(False)
writer = SummaryWriter()
sample_rate = 44100
freqs = [262, 294, 330, 349, 392, 440, 440, 440, 440, 440, 440]

for n_iter in range(100):

    dummy_s1 = torch.rand(1)
    dummy_s2 = torch.rand(1)
    # data grouping by `slash`
    writer.add_scalar('data/scalar1', dummy_s1[0], n_iter)
    writer.add_scalar('data/scalar2', dummy_s2[0], n_iter)

    writer.add_scalars('data/scalar_group', {'xsinx': n_iter * np.sin(n_iter),
                                             'xcosx': n_iter * np.cos(n_iter),
                                             'arctanx': np.arctan(n_iter)}, n_iter)

    dummy_img = torch.rand(32, 3, 64, 64)  # output from network
    if n_iter % 10 == 0:
        x = vutils.make_grid(dummy_img, normalize=True, scale_each=True)
        writer.add_image('Image', x, n_iter)

        dummy_audio = torch.zeros(sample_rate * 2)
        for i in range(x.size(0)):
            # amplitude of sound should in [-1, 1]
            dummy_audio[i] = np.cos(freqs[n_iter // 10] * np.pi * float(i) / float(sample_rate))
        writer.add_audio('myAudio', dummy_audio, n_iter, sample_rate=sample_rate)

        writer.add_text('Text', 'text logged at step:' + str(n_iter), n_iter)

        for name, param in resnet18.named_parameters():
            writer.add_histogram(name, param.clone().cpu().data.numpy(), n_iter)

        # needs tensorboard 0.4RC or later
        writer.add_pr_curve('xoxo', np.random.randint(2, size=100), np.random.rand(100), n_iter)

dataset = datasets.MNIST('mnist', train=False, download=True)
images = dataset.test_data[:100].float()
label = dataset.test_labels[:100]

features = images.view(100, 784)
writer.add_embedding(features, metadata=label, label_img=images.unsqueeze(1))

# export scalar data to JSON for external processing
writer.export_scalars_to_json("./all_scalars.json")
writer.close()

Extracting mnist/MNIST/raw/train-images-idx3-ubyte.gz to mnist/MNIST/raw


Extracting mnist/MNIST/raw/train-labels-idx1-ubyte.gz to mnist/MNIST/raw



Extracting mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist/MNIST/raw


Extracting mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist/MNIST/raw
Processing...
Done!


/home/ali/.virtualenvs/py37/lib/python3.7/site-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/ali/.virtualenvs/py37/lib/python3.7/site-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
